In [42]:
import pandas as pd
df = pd.read_csv("../mails.csv")
df

,Subject,Sender,Sender ID,Body
0,Reminder: Your Google Cloud free trial is waiting,Google Cloud,<CloudPlatform-noreply@google.com>,Go to your Console\nYou’re almost there.\nFini...
1,Request for Zoom Meeting: Team Training,Uddipta Deka,<uddipta222000deka@gmail.com>,"Dear IOCL,\n\nI hope this email finds you in g..."
2,Request for Zoom Meeting: Project Discussion,Uddipta Deka,<uddipta222000deka@gmail.com>,"Dear IOCL,\n\nI hope this email finds you well..."
3,TestMail,Uddipta Deka,<uddipta222000deka@gmail.com>,This mail is sent only for testing purposes.


In [56]:
df['Subject'].iloc[1]

'Request for Zoom Meeting: Team Training'

In [190]:
keywords = ['Zoom','Meeting','Link','Meeting ID','Password','Meeting Password','Meeting Link','Zoom Link','Zoom Meeting']

In [191]:
res = df[df['Body'].str.contains('|'.join(keywords))]
res

,Subject,Sender,Sender ID,Body
1,Request for Zoom Meeting: Team Training,Uddipta Deka,<uddipta222000deka@gmail.com>,"Dear IOCL,\n\nI hope this email finds you in g..."
2,Request for Zoom Meeting: Project Discussion,Uddipta Deka,<uddipta222000deka@gmail.com>,"Dear IOCL,\n\nI hope this email finds you well..."


In [192]:
print(res['Body'].iloc[0])
print(res['Body'].iloc[1])

Dear IOCL,

I hope this email finds you in good health. I am writing to request a Zoom meeting to conduct a team training session on volleyball. As our team continues to grow, it is crucial that we enhance our skills and knowledge in this area to ensure consistent performance and productivity.

I suggest the following details for the Zoom meeting:

Date: July 20, 2023
Time: 2:00 PM (GMT)
Duration: 1.5 hours

Please let me know if the proposed date and time are convenient for you. If not, I am open to alternative suggestions. Your participation in this training will greatly contribute to the development of our team, and I appreciate your support.

Thank you for your attention to this matter. I look forward to your positive response.

Warm regards,
Dear IOCL,

I hope this email finds you well. I would like to request a Zoom meeting to discuss an important project that requires your input and guidance. The meeting will provide an opportunity to go over the project details, address any con

In [193]:
import re
from dateutil import parser
from datetime import datetime, timedelta
len(res)
date_and_time_list = []
duration_list = []
for index in range(len(res)):
    body = res['Body'].iloc[index]
    # date = body[body.find('Date:')+6:body.find('Time:')-1]
    # time = body[body.find('Time:')+6:body.find('Time:')+20]
    date = parser.parse(body, fuzzy=True).date()
    matchTime = re.search(r'Time: (\d{1,2}:\d{2} [AP]M)', body)
    matchDuration = re.search(r'Duration: ([\d.]+) (\w+)', body)
    if matchTime and matchDuration:
        time = matchTime.group(1)
        duration_val = float(matchDuration.group(1))
        duration_unit = matchDuration.group(2)

        if duration_unit == 'hour' or duration_unit == 'hours':
            duration = timedelta(hours=duration_val)
        elif duration_unit == 'minute' or duration_unit == 'minutes':
            duration = timedelta(minutes=duration_val)
        else:
            duration = timedelta()

    date_and_time = datetime.strptime(str(date) + " " + time, '%Y-%m-%d %I:%M %p')

    date_and_time_list.append(date_and_time)
    duration_list.append(duration)

print(date_and_time_list)
# print(time)
print(duration_list)

[datetime.datetime(2023, 7, 20, 14, 0), datetime.datetime(2023, 7, 12, 10, 0)]
[datetime.timedelta(seconds=5400), datetime.timedelta(seconds=3600)]


In [194]:
final_mail = res.drop('Body', axis=1)
final_mail['Date and Time'] = pd.Series(date_and_time_list).tolist()
final_mail['Duration'] = pd.Series(duration_list).tolist()
final_mail

,Subject,Sender,Sender ID,Date and Time,Duration
1,Request for Zoom Meeting: Team Training,Uddipta Deka,<uddipta222000deka@gmail.com>,2023-07-20 14:00:00,0 days 01:30:00
2,Request for Zoom Meeting: Project Discussion,Uddipta Deka,<uddipta222000deka@gmail.com>,2023-07-12 10:00:00,0 days 01:00:00


In [195]:
prompt = 'Extract the date and time for the meeting\n' + str(df['Body'].iloc[1] + 'covert it to the below format\n 2020-12-12 10:00 AM') 
duration_prompt = "Give the Duration of the meeting in minutes after reading the paragraph.\n" + str(res['Body'].iloc[1]) + "\nIf no duration mentioned, just provide a duration in minutes.\nDon't add any other information in the response.(Response should be only the digit)\n"


In [196]:
print(duration_prompt)

Give the Duration of the meeting in minutes after reading the paragraph.
Dear IOCL,

I hope this email finds you well. I would like to request a Zoom meeting to discuss an important project that requires your input and guidance. The meeting will provide an opportunity to go over the project details, address any concerns, and ensure that we are aligned on the next steps.

I propose the following details for the Zoom meeting:

Date: July 12, 2023
Time: 10:00 AM (GMT)
Duration: 1 hour

Please let me know if the suggested date and time work for you, or if you have any alternative suggestions. I believe your expertise and insights will be invaluable to the success of this project, and I look forward to our discussion.

Thank you for considering my request. I appreciate your time and cooperation.

Best regards,
If no duration mentioned, just provide a duration in minutes.
Don't add any other information in the response.(Response should be only the digit)



In [197]:
import os
os.environ["_BARD_API_KEY"] = '[Bard API key]'

In [198]:

from bardapi import Bard
res = Bard().get_answer(duration_prompt)['content']
print(res)
# res = '1 hour and 30 minutes'

The duration of the meeting is 60 minutes.

The email states that the meeting will last for "1 hour". There are 60 minutes in an hour, so the duration of the meeting is 60 minutes.


In [172]:
import re
duration_pattern = r"\d+ hour(?:s)? and \d+ minute(?:s)?|\d+ minute(?:s)?"
# duration_pattern = r"\d+ hour(?:s)? and \d+ minute(?:s)?|\d+ hour(?:s)?"
match = re.search(duration_pattern, res)
if match:
    duration = match.group()
    print(duration)

90 minutes


In [104]:
import spacy
nlp = spacy.load("en_core_web_sm")

result = nlp(res)

# Access the first token
first_token = result[0].text
print(first_token)

Yes
